In [ ]:
import csv
import os
import zipfile

In [ ]:
# Descompactando
#with zipfile.ZipFile("data/dados benignos/logs completos/Messages_Traffic.zip", 'r') as zip_ref:
#    zip_ref.extractall("data/dados benignos/logs completos/")

O tratamento consiste em extrair um formato de planilha onde cada linha tem o [id, data] de cada mensagem CAN.

In [ ]:
if not os.path.exists("data/dados benignos/logs tratados"):
    os.makedirs("data/dados benignos/logs tratados")

lista_zipada = list()

for file in os.listdir("data/dados benignos/logs completos"):

    if file == "Messages_Traffic.zip":
        continue

    timestamp_list = list()
    id_list = list()
    dlc_list = list()
    relative_time_list = list()
    byte_0_list = list()
    byte_1_list = list()
    byte_2_list = list()
    byte_3_list = list()

    with open(f"data/dados benignos/logs completos/{file}", mode='r', errors="ignore") as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        valido = False
        for linha in leitor_csv:
            if "DW CAN 01" in linha:
                for coluna in linha[9:10]:
                    if len(coluna) == 2:
                        id_list.append(coluna)
                        valido = True

                if valido == True:
                    count = 0
                    colunas = int("0", 16)

                    byte_0_flag = False
                    byte_1_flag = False
                    byte_2_flag = False
                    byte_3_flag = False
                    for coluna in linha[12:16]:
                        if coluna != "":
                            count += 1

                            if count == 1:
                                byte_0_list.append(int(str(coluna), 16))
                                byte_0_flag = True
                            elif count == 2:
                                byte_1_list.append(int(str(coluna), 16))
                                byte_1_flag = True
                            elif count == 3:
                                byte_2_list.append(int(str(coluna), 16))
                                byte_2_flag = True
                            elif count == 4:
                                byte_3_list.append(int(str(coluna), 16))
                                byte_3_flag = True

                    if byte_0_flag == False:
                        byte_0_list.append(0)
                    if byte_1_flag == False:
                        byte_1_list.append(0)
                    if byte_2_flag == False:
                        byte_2_list.append(0)
                    if byte_3_flag == False:
                        byte_3_list.append(0)

                    dlc_list.append(count)

                    for coluna in linha[1:2]:
                        timestamp_list.append(coluna)

                    for coluna in linha[2:3]:
                        relative_time_list.append(coluna)

                valido = False

    for identification in id_list:
        id_list[id_list.index(identification)] = int(identification, 16)

    lista_zipada.extend(list(zip(timestamp_list, relative_time_list, dlc_list, id_list, byte_0_list, byte_1_list, byte_2_list, byte_3_list)))

lista_zipada.insert(0, ["Timestamp", "Rel Time", "DLC", "Arb ID", "Byte 0", "Byte 1", "Byte 2", "Byte 3"])

with open(f"data/dados benignos/logs tratados/Treated_Benign_Traffic.csv", mode="w", newline="") as arquivo_csv:
    escritor_csv = csv.writer(arquivo_csv)
    for elemento in lista_zipada:
        escritor_csv.writerow(elemento)